In [1]:
import logging
import sys
import os

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# DB credentials
username="neo4j"
password="qwer1234"
url="bolt://localhost:7687"
database="sample-jobs"

In [3]:
from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph
from langchain_openai import ChatOpenAI

graph = Neo4jGraph(
    url=url, 
    username=username, 
    password=password,
    database=database,
)

In [6]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, model_name="gpt-4-0125-preview")

llm_transformer = LLMGraphTransformer(llm=llm)

In [7]:
from langchain_core.documents import Document

text = """
Marie Curie, was a Polish and naturalised-French physicist and chemist who conducted pioneering research on radioactivity.
She was the first woman to win a Nobel Prize, the first person to win a Nobel Prize twice, and the only person to win a Nobel Prize in two scientific fields.
Her husband, Pierre Curie, was a co-winner of her first Nobel Prize, making them the first-ever married couple to win the Nobel Prize and launching the Curie family legacy of five Nobel Prizes.
She was, in 1906, the first woman to become a professor at the University of Paris.
"""
documents = [Document(page_content=text)]
graph_documents = llm_transformer.convert_to_graph_documents(documents)

print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Nodes:[Node(id='Marie Curie', type='Person'), Node(id='Polish', type='Nationality'), Node(id='Naturalised-French', type='Nationality'), Node(id='Physicist', type='Occupation'), Node(id='Chemist', type='Occupation'), Node(id='Nobel Prize', type='Award'), Node(id='Pierre Curie', type='Person'), Node(id='University Of Paris', type='Organization')]
Relationships:[Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id='Polish', type='Nationality'), type='NATIONALITY'), Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id='Naturalised-French', type='Nationality'), type='NATIONALITY'), Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id='Physicist', type='Occupation'), type='OCCUPATION'), Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id='C

In [9]:
for node in graph_documents[0].nodes:
    print(node)

id='Marie Curie' type='Person'
id='Polish' type='Nationality'
id='Naturalised-French' type='Nationality'
id='Physicist' type='Occupation'
id='Chemist' type='Occupation'
id='Nobel Prize' type='Award'
id='Pierre Curie' type='Person'
id='University Of Paris' type='Organization'


In [10]:
for relationship in graph_documents[0].relationships:
    print(relationship)

source=Node(id='Marie Curie', type='Person') target=Node(id='Polish', type='Nationality') type='NATIONALITY'
source=Node(id='Marie Curie', type='Person') target=Node(id='Naturalised-French', type='Nationality') type='NATIONALITY'
source=Node(id='Marie Curie', type='Person') target=Node(id='Physicist', type='Occupation') type='OCCUPATION'
source=Node(id='Marie Curie', type='Person') target=Node(id='Chemist', type='Occupation') type='OCCUPATION'
source=Node(id='Marie Curie', type='Person') target=Node(id='Nobel Prize', type='Award') type='AWARD_WINNER'
source=Node(id='Pierre Curie', type='Person') target=Node(id='Nobel Prize', type='Award') type='AWARD_WINNER'
source=Node(id='Marie Curie', type='Person') target=Node(id='University Of Paris', type='Organization') type='PROFESSOR'


In [11]:
llm_transformer_filtered = LLMGraphTransformer(
    llm=llm,
    # allowed_nodes=["Person", "Country", "Organization"],
    # allowed_relationships=["NATIONALITY", "LOCATED_IN", "WORKED_AT", "SPOUSE"],
)
graph_documents_filtered = llm_transformer_filtered.convert_to_graph_documents(


    documents
)\

print(f"Nodes:{graph_documents_filtered[0].nodes}")
print(f"Relationships:{graph_documents_filtered[0].relationships}")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Nodes:[Node(id='Marie Curie', type='Person'), Node(id='Pierre Curie', type='Person'), Node(id='University Of Paris', type='Organization'), Node(id='Poland', type='Country'), Node(id='France', type='Country')]
Relationships:[Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id='Poland', type='Country'), type='NATIONALITY'), Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id='France', type='Country'), type='NATIONALITY'), Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id='University Of Paris', type='Organization'), type='WORKED_AT'), Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id='Pierre Curie', type='Person'), type='SPOUSE')]


In [14]:
#한국어 테스트

text = """
최근 삼성, LG, 한화, 두산 등 굴지의 대기업부터 소규모 스타트업까지 '자동조리로봇' 시장에 앞다퉈 뛰어들고 있다. 자동조리로봇이 저출산·고임금으로 주방 구인난에 시달리는 외식업체들의 '해결사'로 지목되면서 관련 시장이 폭발적으로 성장할 전망이다. 이에 여러 로봇업체들이 진출해 경쟁을 펼치고 있지만 아직 초기 단계여서 시장을 리드하는 업체는 나타나지 않은 상황이다. 이들 업체들은 외식 산업의 수요를 넘어 집집마다 부엌에서 쓰일 수 있는 조리로봇을 개발해 시장을 글로벌 무대로 확대하겠다는 포부를 갖고 있다. 이들이 개발하는 자동조리로봇은 완벽한 '주방 자동화'를 이뤄낼 수 있을까?
"""
ko_documents = [Document(page_content=text)]
ko_graph_documents = llm_transformer.convert_to_graph_documents(ko_documents)

print(f"Nodes:{ko_graph_documents[0].nodes}")
print(f"Relationships:{ko_graph_documents[0].relationships}")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Nodes:[Node(id='Samsung', type='Company'), Node(id='Lg', type='Company'), Node(id='Hanwha', type='Company'), Node(id='Doosan', type='Company'), Node(id='Small Startups', type='Company'), Node(id='Automatic Cooking Robots', type='Product'), Node(id='Restaurant Industry', type='Industry'), Node(id='Global Market', type='Market')]
Relationships:[Relationship(source=Node(id='Samsung', type='Company'), target=Node(id='Automatic Cooking Robots', type='Product'), type='ENTERS'), Relationship(source=Node(id='Lg', type='Company'), target=Node(id='Automatic Cooking Robots', type='Product'), type='ENTERS'), Relationship(source=Node(id='Hanwha', type='Company'), target=Node(id='Automatic Cooking Robots', type='Product'), type='ENTERS'), Relationship(source=Node(id='Doosan', type='Company'), target=Node(id='Automatic Cooking Robot

In [ ]:
# graph.add_graph_documents(graph_documents_filtered)